### Sentiment anlayis for the Reviews



<b> Importing needed Libaries

In [3]:
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
import string
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer

#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')


<b>Creating Data Frame with the csv file

In [4]:

filePath = '../dataset/Reviews only (Sentiment) only 20.csv'
df = pd.read_csv(filePath, delimiter='\t')
df

,reviewText
0,I've been using these for over a year now. Ba...
1,I have owned these for over two years and wore...
2,I bought these thinking they were the ticket t...
3,I've owned these for years. I used to fly two ...
4,I love these headphones!! I got these in Narit...
5,Work Good. I don't have another reference. You...
6,This headphone was my first foray into noise-r...
7,Not as immersive and noise-cancelling as other...
8,I own these and the Bose Quiet Comfort 2 ($250...
9,Sound quality of the headset (noise canceling ...


<b>Using VADER to analayze the sentiment of the reviews


In [5]:
# To get the compound score
def sentimental(sentence):
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(sentence)
    score=vs['compound']
    return score

# VADER prediting Positive negative netural
def sentimental_Score(sentence):
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(sentence)
    score=vs['compound']
    if score >= 0:
        return 'pos'
    else:
        return 'neg'

In [6]:
df.head()

,reviewText
0,I've been using these for over a year now. Ba...
1,I have owned these for over two years and wore...
2,I bought these thinking they were the ticket t...
3,I've owned these for years. I used to fly two ...
4,I love these headphones!! I got these in Narit...


In [7]:
#!pip3 install sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

In [11]:
# Selected_Rows = df.head(10)
#df.insert(1, "lab",0)

df["compound"]=df['reviewText'].apply(lambda x: sentimental_Score(x))
df



,reviewText,compound
0,I've been using these for over a year now. Ba...,pos
1,I have owned these for over two years and wore...,pos
2,I bought these thinking they were the ticket t...,pos
3,I've owned these for years. I used to fly two ...,pos
4,I love these headphones!! I got these in Narit...,pos
5,Work Good. I don't have another reference. You...,pos
6,This headphone was my first foray into noise-r...,pos
7,Not as immersive and noise-cancelling as other...,pos
8,I own these and the Bose Quiet Comfort 2 ($250...,pos
9,Sound quality of the headset (noise canceling ...,pos


In [12]:
# Using SVM classifier on the train set
from sklearn.svm import LinearSVC
# classifer = LinearSVC(max_iter=df.shape[0])

In [13]:
tfidf_lr_pipe = Pipeline([('tfidf', TfidfVectorizer(lowercase=False)), ('svm', LinearSVC(max_iter=df.shape[0]))])

In [14]:
df['reviewText']=df['reviewText'].apply(str)
df['compound']=df['compound'].apply(str)

In [15]:
x = df.iloc[:,0].values
x[0]

"I've been using these for over a year now.  Basically, in a quiet environment these sound pretty good.  They aren't as good as similarly-priced studio-class headsets, but you're paying for the noise cancelling.  The raw capability is pretty decent, and has acceptable detail in treble, vocal, and bass ranges.\n\nThe noise cancelling works pretty well.  I've taken these on several intercontinental flights and they do a decent job of blocking out engine rumble and screaming kids.  They also do a decent job of blocking out adult human voices, which is nice in the office.\n\nHowever, I must have pretty big ears, and I've found that the foam cups aren't quite thick enough to adequately push the driver away from my outer ear.  After 2 or 3 hours of use my ears start to ache a bit.\n\nThe cord is plenty long, and can be disconnected in the middle to link to an optional Sony wired remote control.  An adapter to convert the standard 1/8th inch stereo jack to the dual 1/8th inch jacks in many ai

In [16]:
y = df.iloc[:,1].values
y[0]

'pos'

In [17]:
# Splitting the train and test data (80/20)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [18]:
# Converting the reviews to a matrix of token counts (Bag of words model)
tfidf_lr_pipe.fit(x_train, y_train)

/home/joker/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Pipeline(steps=[('tfidf', TfidfVectorizer(lowercase=False)),
                ('svm', LinearSVC(max_iter=19))])

In [19]:
tfidf_lr_pipe.fit(x_train, y_train)

/home/joker/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Pipeline(steps=[('tfidf', TfidfVectorizer(lowercase=False)),
                ('svm', LinearSVC(max_iter=19))])

In [20]:
y_pred = tfidf_lr_pipe.predict(x_test)

In [21]:
predicted_df = pd.DataFrame({'predict': y_pred, 'actual': y_test})

In [22]:
predicted_df.head(100)


,predict,actual
0,pos,pos
1,pos,pos
2,pos,pos
3,pos,pos


In [25]:
# Confusion matrix to see the number of true positive, true negative, false positive and false negative
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[4]])

In [55]:
import pickle
filename = "../ML models/serialized_pipeline_model_tfid.pkl"  

with open(filename, 'wb') as file:  
    pickle.dump(tfidf_lr_pipe, file)